<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Clearing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "git+https://github.com/Karasiari/HuGraphFinal.git"

  Cloning https://github.com/Karasiari/HuGraphFinal.git to /tmp/pip-req-build-4ai0p74d
  Running command git clone --filter=blob:none --quiet https://github.com/Karasiari/HuGraphFinal.git /tmp/pip-req-build-4ai0p74d
  Resolved https://github.com/Karasiari/HuGraphFinal.git to commit d8eed02799a883483f06a8de6fd5364ad81ade54
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hugraphfin: filename=hugraphfin-0.1.0-py3-none-any.whl size=64514 sha256=02755a8a5d6cabd6ddc0f8f6eb4acad087124deadce7262b68bca97c23114616
  Stored in directory: /tmp/pip-ephem-wheel-cache-g9zbyfyu/wheels/46/83/89/572a6db8e6320eb877bf6ced236fa61f993538a12c5e43c72e
Successfully built hugraphfin


In [2]:
# чтобы не падал ecos
%pip install ecos

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 4.2 MB/s eta 0:00:00


In [3]:
# откуда читать
from google.colab import files
path_to_folder = '/content/drive/MyDrive/Кола'

In [ ]:
# импорт читалки ху графов
from graph_exps.readers.hugraph_reader import read_hu_graphs
# импорт генерации своего трафика
from graph_exps.traffic_generation.generation_for_exp import generate_own_traffic
# импорт генераци SNR-BA графа
from graph_exps.SNRBA_generation.generation import generate_snr_ba_graph
# импорт окружения эксперимента
from graph_exps.exp import *

# допустимые веса запросов и capacity для SNR-BA графа
available_demand_volumes = ((1, 0.9), (2, 0.1))
capacity_value = 80.0

# новые ресурсы и типы для распределения ресурсов
additional_resources = [capacity_value] * 3
allocation_types = ['alpha', 'random_alpha', 'random']

# генерация SNR-BA графа
snr_ba_graph = generate_snr_ba_graph(num_nodes=30, capacity_value=capacity_value)

# генерация своего трафика методом "alpha_with_sa" и эксперимент для графа
adj_graph_alpha, traffic_graph_alpha =  generate_own_traffic(
  snr_ba_graph,
  available_demand_volumes,
  generation_type="alpha_with_sa",
  generation_params={"alpha_target": 0.35, "epsilon": 0.025, "median_weight_for_initial": 20, "var_for_initial": 1, "multi_max": 5, "t": 0.5}
)
graph_for_exp_alpha = HuGraphForExps(adj_graph_alpha, traffic_graph_alpha)
results_alpha = expand_test_for_graph(
  graph=graph_for_exp_alpha,
  additional_resources=additional_resources,
  allocation_types=allocation_types,
  tries_for_allocation=10,
  epsilon=1.2,
  available_volumes=available_demand_volumes
)

# генерация своего трафика методом "gravity" и эксперимент для графа
adj_graph_gravity, traffic_graph_gravity =  generate_own_traffic(
  snr_ba_graph,
  available_demand_volumes,
  generation_type="gravity",
  generation_params={"beta": 0.15, "dyn_k": 0.9}
)
graph_for_exp_gravity = HuGraphForExps(adj_graph_gravity, traffic_graph_gravity)
results_gravity = expand_test_for_graph(
  graph=graph_for_exp_gravity,
  additional_resources=additional_resources,
  allocation_types=allocation_types,
  tries_for_allocation=10,
  epsilon=1.2,
  available_volumes=available_demand_volumes
)


# то же, но для ху графа

# читаем граф
hu_graphs_names = ['cola_t3']
hu_graphs = read_hu_graphs(path_to_folder, hu_graphs_names, True)
adj_graph_hu, traffic_graph_hu = hu_graphs['cola_t3']['adj_graph'], hu_graphs['cola_t3']['traffic_graph']

# проводим эксперимент
graph_for_exp_hu = HuGraphForExps(adj_graph_hu, traffic_graph_hu)
results_hu = expand_test_for_graph(
  graph=graph_for_exp_hu,
  additional_resources=additional_resources,
  allocation_types=allocation_types,
  tries_for_allocation=10,
  epsilon=1.2,
  available_volumes=available_demand_volumes
)

In [5]:
results_alpha

allocation solved rerouted volume ratio  \
alpha        1              False                  0.56   
             2              False                  0.06   
             3              False                  0.26   
             4              False                  0.47   
             5              False                  0.15   
             6              False                  0.24   
             7              False                  0.28   
             8              False                  0.06   
             9              False                  0.01   
             10             False                  0.56   
random_alpha 1              False                  0.18   
             2              False                  0.04   
             3              False                  0.04   
             4              False                  0.24   
             5              False                  0.07   
             6              False                  0.02   
             7              False                  0.07   
             8              False                  0.22   
             9              False                  0.07   
             10             False                  0.04   
random       1              False                  0.33   
             2              False                  0.13   
             3              False                  0.07   
             4              False                  0.04   
             5              False                   0.1   
             6              False                  0.21   
             7              False                  0.26   
             8              False                  0.08   
             9              False                  0.06   
             10             False                  0.06   

                gamma for failed (0, 14) gamma for failed (0, 19)  \
alpha        1                      2.19                     3.97   
             2                      2.19                     3.97   
             3                      2.19                     3.97   
             4                      2.19                     3.97   
             5                      2.19                     3.97   
             6                      2.19                     3.97   
             7                      2.19                     3.97   
             8                      2.19                     3.97   
             9                      2.19                     3.97   
             10                     2.19                     3.97   
random_alpha 1                      1.39                      1.8   
             2                      1.39                      1.8   
             3                      1.39                      1.8   
             4                      1.39                      1.8   
             5                      1.39                      1.8   
             6                      1.39                      1.8   
             7                      1.39                      1.8   
             8                      1.39                      1.8   
             9                      1.39                      1.8   
             10                     1.39                      1.8   
random       1                      0.67                     1.21   
             2                      0.67                     1.21   
             3                      0.67                     1.21   
             4                      0.67                     1.21   
             5                      0.67                     1.21   
             6                      0.67                     1.21   
             7                      0.67                     1.21   
             8                      0.67                     1.21   
             9                      0.67                     1.21   
             10                     0.67                     1.21   

                gamma for failed (1, 10) gamma

In [6]:
results_gravity

allocation solved rerouted volume ratio  \
alpha        1               True                  1.19   
             2               True                  1.19   
             3              False                  0.29   
             4              False                  0.52   
             5               True                  1.19   
             6               True                  1.19   
             7               True                  1.19   
             8               True                  1.19   
             9              False                  0.52   
             10             False                  0.14   
random_alpha 1               True                  1.19   
             2               True                  1.19   
             3               True                  1.18   
             4               True                  1.18   
             5               True                  1.18   
             6              False                  0.58   
             7              False                  0.77   
             8               True                  1.18   
             9               True                  1.18   
             10              True                  1.18   
random       1              False                  0.17   
             2              False                  0.31   
             3              False                   0.1   
             4              False                  0.62   
             5              False                  0.73   
             6               True                  1.19   
             7               True                  1.19   
             8               True                  1.19   
             9               True                  1.19   
             10              True                  1.19   

                gamma for failed (0, 14) gamma for failed (0, 19)  \
alpha        1                      2.12                     3.26   
             2                      2.12                     3.26   
             3                      2.12                     3.26   
             4                      2.12                     3.26   
             5                      2.12                     3.26   
             6                      2.12                     3.26   
             7                      2.12                     3.26   
             8                      2.12                     3.26   
             9                      2.12                     3.26   
             10                     2.12                     3.26   
random_alpha 1                      2.23                      3.2   
             2                      2.23                      3.2   
             3                      2.23                      3.2   
             4                      2.23                      3.2   
             5                      2.23                      3.2   
             6                      2.23                      3.2   
             7                      2.23                      3.2   
             8                      2.23                      3.2   
             9                      2.23                      3.2   
             10                     2.23                      3.2   
random       1                      2.48                     2.75   
             2                      2.48                     2.75   
             3                      2.48                     2.75   
             4                      2.48                     2.75   
             5                      2.48                     2.75   
             6                      2.48                     2.75   
             7                      2.48                     2.75   
             8                      2.48                     2.75   
             9                      2.48                     2.75   
             10                     2.48                     2.75   

                gamma for failed (1, 10) gamma

In [7]:
results_hu

allocation solved rerouted volume ratio  \
alpha        1              False                  0.22   
             2              False                  0.17   
             3              False                  0.34   
             4              False                  0.29   
             5              False                  0.34   
             6              False                  0.35   
             7              False                  0.47   
             8              False                  0.98   
             9              False                  0.37   
             10             False                  0.11   
random_alpha 1              False                  0.58   
             2              False                   0.8   
             3              False                  0.07   
             4              False                  0.62   
             5              False                  0.32   
             6              False                  0.54   
             7              False                  0.07   
             8              False                  0.54   
             9              False                  0.85   
             10             False                  0.78   
random       1              False                  0.46   
             2              False                   0.4   
             3              False                   0.4   
             4              False                  0.76   
             5              False                  0.47   
             6              False                  0.55   
             7              False                  0.19   
             8              False                  0.13   
             9              False                   0.1   
             10             False                   0.7   

                gamma for failed (0, 15) gamma for failed (0, 4)  \
alpha        1                      1.18                    13.4   
             2                      1.18                    13.4   
             3                      1.18                    13.4   
             4                      1.18                    13.4   
             5                      1.18                    13.4   
             6                      1.18                    13.4   
             7                      1.18                    13.4   
             8                      1.18                    13.4   
             9                      1.18                    13.4   
             10                     1.18                    13.4   
random_alpha 1                      2.18                   10.83   
             2                      2.18                   10.83   
             3                      2.18                   10.83   
             4                      2.18                   10.83   
             5                      2.18                   10.83   
             6                      2.18                   10.83   
             7                      2.18                   10.83   
             8                      2.18                   10.83   
             9                      2.18                   10.83   
             10                     2.18                   10.83   
random       1                      1.14                    7.48   
             2                      1.14                    7.48   
             3                      1.14                    7.48   
             4                      1.14                    7.48   
             5                      1.14                    7.48   
             6                      1.14                    7.48   
             7                      1.14                    7.48   
             8                      1.14                    7.48   
             9                      1.14                    7.48   
             10                     1.14                    7.48   

                gamma for failed (0, 12) gamma for failed (0, 13)  \
alpha   